# Imports

In [34]:
from pathlib import Path
persist_path = Path('.') / 'persist'
import sys
project_root = str(Path(sys.path[0]).parents[0].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
from IPython.display import display
from bokeh.io import output_notebook, show
output_notebook()
from functools import partial
import scripts.utils as utils
from importlib import reload

Loading BokehJS ...

# Acquisition des données

In [42]:
orders = pd.read_pickle(persist_path / 'orders.pkl')
order_count = len(orders)
df_clt = pd.read_pickle(persist_path / 'clt.pkl')
clt_fields = ['seg1', 'seg2', 'seg3', 'seg4', 'cat', 'sscat', 'hier4', 'groupecompte', 'relationtype', 'pilcom']
for field in clt_fields:
    try:
        orders = orders.drop(field)
    except: 
        pass
orders = orders.merge(df_clt.loc[:, clt_fields], left_on=['orgacom', 'client'], right_index=True, validate='m:1')
if len(orders) != order_count:
    raise RuntimeError('Something went wrong when merging orders with client master data!')
    
full_orders = pd.read_pickle(persist_path / 'full_orders_with_zscore.pkl')

# Bar plots

In [26]:
show(partial(utils.bk_histo_seg,
             source_df=orders.reset_index(),
             segs=['seg1', 'seg2'],
             filters={},
             filters_exclude={'groupecompte': ['ZPER']}, # 'hier4': ['198863']
             plot_kwargs=dict(title="Chiffre d'affaire brut 2019 en fonction de la segmentation niveau 2 et du canal"),
            ),
     notebook_url="http://devdm:8888")

In [28]:
show(partial(utils.bk_histo_seg,
             source_df=orders.reset_index(),
             segs=['seg2', 'seg3'],
             filters={'seg1': ['Z3']},
             filters_exclude={'groupecompte': ['ZPER']}, # 'hier4': ['198863']
             plot_kwargs=dict(title="Chiffre d'affaire brut 2019 en fonction de la segmentation niveau 3 et du canal"),
            ),
     notebook_url="http://devdm:8888")

# Bubbles

In [31]:
orders

margin  brutrevenue   weight  linecount  \
orgacom date       client                                                
1SOU    2017-07-03 0000065940   31.16        84.01    5.501          9   
        2017-08-28 0000065940   44.36       105.93    8.048         10   
        2017-09-08 0000065940   17.60        40.35    3.000          2   
        2017-09-15 0000065940  100.12       236.77   20.312          3   
        2017-09-25 0000065940   96.04       253.99   40.338          5   
...                               ...          ...      ...        ...   
2BRE    2020-08-31 0000282373  177.42      1401.58  488.036         88   
                   0000283953   43.79       421.67  564.980         28   
                   0000284089   89.39       186.75   90.200         12   
                   0000284540  262.04      1252.07  639.326         97   
                   0000284735  319.24       782.56  235.236         58   

                              origin2 seg1 seg2 seg3 seg4 cat sscat  hier4  \
orgacom date       client                                                    
1SOU    2017-07-03 0000065940      TV   Z3   Z4   ZI   ZB  ZL   Z13  10897   
        2017-08-28 0000065940      TV   Z3   Z4   ZI   ZB  ZL   Z13  10897   
        2017-09-08 0000065940      TV   Z3   Z4   ZI   ZB  ZL   Z13  10897   
        2017-09-15 0000065940      TV   Z3   Z4   ZI   ZB  ZL   Z13  10897   
        2017-09-25 0000065940      TV   Z3   Z4   ZI   ZB  ZL   Z13  10897   
...                               ...  ...  ...  ...  ...  ..   ...    ...   
2BRE    2020-08-31 0000282373     EDI   Z3   Z4   ZJ   ZF  ZP    ZA  10705   
                   0000283953     EDI   Z3   Z4   ZJ   ZF  ZN   Z19  10506   
                   0000284089      TV   Z3   Z5   ZK   ZG  ZU   Z43    NaN   
                   0000284540      TV   Z3   Z4   ZJ   ZF  ZP    ZA  10432   
                   0000284735      VR   Z3   Z5   ZK   ZG  ZU    ZN    NaN   

                              groupecompte relationtype pilcom  
orgacom date       client                                       
1SOU    2017-07-03 0000065940         ZCLT          NDI    RGL  
        2017-08-28 0000065940         ZCLT          NDI    RGL  
        2017-09-08 0000065940         ZCLT          NDI    RGL  
        2017-09-15 0000065940         ZCLT          NDI    RGL  
        2017-09-25 0000065940         ZCLT          NDI    RGL  
...                                    ...          ...    ...  
2BRE    2020-08-31 0000282373         ZCRC          DIR    BIG  
                   0000283953         ZCRC          DIR    NTL  
                   0000284089         ZCLT          NDI    RGL  
                   0000284540         ZCRC          DIR    NTL  
                   0000284735         ZCHN          NDI    RGL  

[2444326 rows x 15 columns]

In [83]:
reload(utils)

<module 'scripts.utils' from '/home/pmasse/WebOrNot/scripts/utils.py'>

In [84]:
filter_rhd = (full_orders.seg1 == 'Z3')
filter_main_origins = ~(
    (full_orders.origin2.isin(['EDI']) & full_orders.seg3.isin(['ZI', 'ZK'])) |
    (full_orders.origin2.isin(['VR']) & full_orders.seg3.isin(['ZJ', 'ZL'])) |
    (full_orders.origin2.isin(['WEB']) & full_orders.seg3.isin(['ZJ']))
)
# filters = filter_rhd & filter_main_origins & filter_ppf
begin_date, end_date = pd.to_datetime('2017-01-01'), pd.to_datetime('2022-12-31')
show(partial(utils.bk_bubbles,
             data=full_orders,
             filters=filter_rhd & filter_main_origins,
#              compute_zscores=True,
            ),
     notebook_url="http://devdm:8888")